# 💡 MosaicX: **"Partitioning, sensitivity, and uncertainty propagation"**

> All code and examples are shared to help researchers, students, and engineers understand the reasoning behind DDDA — and to make it easy to apply dimensional analysis to your own data.  
> This notebook serves as an entry-level guide for teaching, validating physical models, and enabling domain-specific knowledge engineering through data-driven dimensional reasoning.

---

## 🎯 What You'll Learn

**隐函数最优显式化 - 机器科学家应用**

In this notebook, we will walk through the theoretical and computational foundation of **dimensional analysis**, with a focus on the **Buckingham Pi theorem**. You will learn:

1. **物理模型，隐函数，流形**  
   Understand why we reduce variables and how dimensional consistency enables model generalization.

2. **变量组合**  
   Encode physical units of input quantities using base units and build the D-matrix.

3. **变量组合评估**  
   Discover dimensionless groups by solving linear algebraic equations on the D-matrix.

4. **显式化策略可视化**  
   Learn to assess whether derived groups make physical and computational sense.

5. **不确定性定量化**  
   Set the stage for further steps in the DDDA pipeline including Pi-group selection, uncertainty quantification, and regime detection.

---

## 👤 Author

- **Name**: Jiashun Pang  
- **Created**: August 2025  
- **Affiliation**: DDDA Project, open research notebook  
- **Notebook Focus**:  
  A hands-on exploration of dimensional analysis — from aggregated raw quantities to symbolic Pi-group discovery and preparation for downstream DDDA tasks.

---

📌 *This notebook is designed to be accessible for learners new to dimensional analysis, while also laying the foundation for advanced applications in the full MosaicPi pipeline.*


---

**Jacobian 矩阵的条件数(Condition Number)** $det(J_y)$的计算与数值检测（含阈值判断）

在数值分析里它用来衡量矩阵的**数值稳定性**和**可逆性好坏**。

在隐函数、力学计算等场景中，cond(J) 是判断**局部解敏感性**的重要指标。

## 1. 定义

对于一个矩阵 $J$，其条件数（通常是 2-范数意义下）定义为：

$$
\mathrm{cond}(J) = \|J\|_2 \cdot \|J^{-1}\|_2
$$

如果 $J$ 是方阵，这个公式也可以写成：

$$
\mathrm{cond}(J) = \frac{\sigma_{\max}}{\sigma_{\min}}
$$

其中：

* $\sigma_{\max}$ 是最大奇异值（矩阵拉伸空间的最大比例）
* $\sigma_{\min}$ 是最小奇异值（最小拉伸比例）

---

## 2. 物理/几何意义

* $J$ 描述了输入空间 $\mathbf{x}$ 到输出空间 $\mathbf{F}(\mathbf{x})$ 的局部线性映射。
* $\mathrm{cond}(J)$ 衡量这个映射的**各方向拉伸比的悬殊程度**：

  * **cond ≈ 1**：各方向缩放比例差不多，映射接近等距，数值稳定。
  * **cond 很大**：某些方向几乎被压扁（σ\_min 很小），说明这个方向上的信息会被放大成很大的数值误差，矩阵接近奇异。

---

## 3. 在隐函数问题中的作用

在隐函数定理应用中：

* 我们通常关心 $J_y = \frac{\partial \mathbf{F}}{\partial \mathbf{y}}$。
* 当 $\mathrm{cond}(J_y)$ 很小时（接近 1），说明局部可逆且数值计算稳定。
* 当 $\mathrm{cond}(J_y)$ 很大时，即使 $\det(J_y) \neq 0$，数值上也会表现为**病态可逆**，容易出现大误差或不稳定解。
* 当 $\mathrm{cond}(J_y) \to \infty$ 时，σ\_min → 0，矩阵奇异，局部不可逆，通常意味着分叉点、临界状态。

---

## 4. 力学中的类比

条件数在力学上很像**刚度矩阵的病态性**：

* **cond 小** ⇒ 各自由度的刚度平衡，加载方向都可控。
* **cond 大** ⇒ 存在软模态（soft mode），某个方向的刚度很小，系统对该方向的扰动非常敏感，可能进入失稳/屈曲。

---

## 6. 常用判断标准

| cond(J) 大小       | 数值意义 | 物理解释（力学）  |
| ---------------- | ---- | --------- |
| $\approx 1$      | 非常稳定 | 方向各向同性    |
| $10^1 \sim 10^3$ | 可接受  | 稍有方向差异    |
| $10^4 \sim 10^8$ | 数值病态 | 存在软模态或强耦合 |
| $\to \infty$     | 奇异矩阵 | 不可逆、分叉点   |

---




In [ ]:
# --- Step 5: 计算条件数 cond(Jy) ---

cond_Jy = np.linalg.cond(Jy_num)
print(f"\ncond(Jy) = {cond_Jy:.6e}")


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- Step 0: knobs ---
N = 300
g = 9.81
v0_min, v0_max = 5.0, 60.0
th_min_deg, th_max_deg = 5.0, 85.0
seed = 42
tol = 1e-3
top_k = 10

# --- Step 1: data ---
rng = np.random.default_rng(seed)
v0 = rng.uniform(v0_min, v0_max, size=N)
th = rng.uniform(th_min_deg, th_max_deg, size=N)
theta = np.deg2rad(th)

R = (v0**2) * np.sin(2 * theta) / g
H = (v0**2) * (np.sin(theta)**2) / (2 * g)

df = pd.DataFrame({
    "v0_mps": v0,
    "theta_deg": th,
    "R_m": R,
    "H_m": H,
    "g_mps2": g
})

# --- Step 2: Jacobian ---
dR_dv0    = (2*v0/g)    * np.sin(2*theta)
dR_dtheta = (2*v0**2/g) * np.cos(2*theta)
dH_dv0    = (v0/g)      * (np.sin(theta)**2)
dH_dtheta = (v0**2/g)   * np.sin(theta) * np.cos(theta)

jacobians = np.array([
    [[dR_dv0[i], dR_dtheta[i]],
     [dH_dv0[i], dH_dtheta[i]]]
    for i in range(N)
])

# --- Step 3: SVD metrics + 符号信息 ---
sigma_min, sigma_max, cond_number, det_val, det_sign = [], [], [], [], []
for J in jacobians:
    _, S, _ = np.linalg.svd(J)
    smin, smax = S.min(), S.max()
    sigma_min.append(smin)
    sigma_max.append(smax)
    cond_number.append(smax/smin if smin > 1e-12 else np.inf)
    d = np.linalg.det(J)
    det_val.append(d)
    det_sign.append(np.sign(d))

df["sigma_min"] = sigma_min
df["sigma_max"] = sigma_max
df["cond_num"]  = cond_number
df["invertible"] = df["sigma_min"] >= tol
df["det_J"] = det_val
df["sign_det"] = det_sign   # -1, 0, +1

# --- Step 4: 可视化 ---
plt.figure(figsize=(6.4,5.4))
# det > 0 用圆点，det < 0 用三角
mask_pos = df["sign_det"] > 0
mask_neg = df["sign_det"] < 0

sc1 = plt.scatter(df.loc[mask_pos,"R_m"], df.loc[mask_pos,"H_m"],
                  c=df.loc[mask_pos,"log10_cond"] if "log10_cond" in df else np.log10(np.clip(df.loc[mask_pos,"cond_num"],1,1e12)),
                  cmap="viridis", s=25, marker="o", label="det>0")

sc2 = plt.scatter(df.loc[mask_neg,"R_m"], df.loc[mask_neg,"H_m"],
                  c=df.loc[mask_neg,"log10_cond"] if "log10_cond" in df else np.log10(np.clip(df.loc[mask_neg,"cond_num"],1,1e12)),
                  cmap="viridis", s=25, marker="^", label="det<0")

plt.colorbar(sc1, label="log10 κ(J)")
plt.xlabel("Range R [m]")
plt.ylabel("Height H [m]")
plt.title("Conditioning & orientation (sign det)")
plt.legend()
plt.show()

# --- Step 5: 打印符号变化点 ---
sign_changes = df[df["sign_det"].diff().fillna(0) != 0]
print("\n⚠️ Points where sign(det(J)) changes (potential branch switch):")
print(sign_changes[["v0_mps","theta_deg","R_m","H_m","det_J","sign_det"]].head(15))


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- Step 0: knobs ---
N = 300
g = 9.81
v0_min, v0_max = 5.0, 60.0
th_min_deg, th_max_deg = 5.0, 85.0
seed = 42
tol = 1e-3                # SVD 最小奇异值阈值
top_k = 10                # 列出最病态的前 K 个样本

# --- Step 1: data ---
rng = np.random.default_rng(seed)
v0 = rng.uniform(v0_min, v0_max, size=N)
th = rng.uniform(th_min_deg, th_max_deg, size=N)
theta = np.deg2rad(th)

R = (v0**2) * np.sin(2 * theta) / g
H = (v0**2) * (np.sin(theta)**2) / (2 * g)

df = pd.DataFrame({
    "v0_mps": v0,
    "theta_deg": th,
    "R_m": R,
    "H_m": H,
    "g_mps2": g
})

# --- Step 2: Jacobian ---
dR_dv0    = (2*v0/g)    * np.sin(2*theta)
dR_dtheta = (2*v0**2/g) * np.cos(2*theta)
dH_dv0    = (v0/g)      * (np.sin(theta)**2)
dH_dtheta = (v0**2/g)   * np.sin(theta) * np.cos(theta)

jacobians = np.array([
    [[dR_dv0[i], dR_dtheta[i]],
     [dH_dv0[i], dH_dtheta[i]]]
    for i in range(N)
])

# --- Step 3: SVD metrics ---
sigma_min, sigma_max, cond_number = [], [], []
for J in jacobians:
    _, S, _ = np.linalg.svd(J)
    smin, smax = S.min(), S.max()
    sigma_min.append(smin)
    sigma_max.append(smax)
    cond_number.append(smax/smin if smin > 1e-12 else np.inf)

df["sigma_min"] = sigma_min
df["sigma_max"] = sigma_max
df["cond_num"]  = cond_number
df["invertible"] = df["sigma_min"] >= tol

# 为了可视化，做 log10(κ)；inf -> 用一个上限替代
cond_cap = 1e12
cond_for_plot = np.clip(np.asarray(cond_number, dtype=float), 1.0, cond_cap)
df["log10_cond"] = np.log10(cond_for_plot)

# --- Step 4: 位置可视化（在输出空间 (R,H)） ---
plt.figure(figsize=(6.2,5.2))
sc1 = plt.scatter(df["R_m"], df["H_m"], c=df["log10_cond"], s=25)
plt.colorbar(sc1, label="log10 κ(J)")
plt.xlabel("Range R [m]")
plt.ylabel("Height H [m]")
plt.title("Conditioning map in output space (R,H)")

# 高亮不可逆点（σ_min < tol）
ill = ~df["invertible"]
plt.scatter(df.loc[ill, "R_m"], df.loc[ill, "H_m"], marker="x", s=40)

plt.show()

# --- Step 5: 位置可视化（在输入空间 (v0, theta)） ---
plt.figure(figsize=(6.2,5.2))
sc2 = plt.scatter(df["v0_mps"], df["theta_deg"], c=df["log10_cond"], s=25)
plt.colorbar(sc2, label="log10 κ(J)")
plt.xlabel("v0 [m/s]")
plt.ylabel("theta [deg]")
plt.title("Conditioning map in input space (v0, θ)")
# 高亮不可逆点
plt.scatter(df.loc[ill, "v0_mps"], df.loc[ill, "theta_deg"], marker="x", s=40)
plt.show()

# --- Step 6: 标注 Top-K 最病态点（在 (R,H) 平面做编号） ---
df_sorted = df.sort_values("cond_num", ascending=False).reset_index(drop=False)
topk = df_sorted.head(top_k)

plt.figure(figsize=(6.6,5.2))
sc3 = plt.scatter(df["R_m"], df["H_m"], c=df["log10_cond"], s=20)
plt.colorbar(sc3, label="log10 κ(J)")
plt.xlabel("Range R [m]")
plt.ylabel("Height H [m]")
plt.title(f"Top-{top_k} worst-conditioned points (numbered)")

for idx, row in topk.iterrows():
    i = int(row["index"])   # 原始索引
    plt.text(row["R_m"], row["H_m"], str(idx+1),
             fontsize=8, ha="center", va="center")

plt.show()

# --- Step 7: 明细与导出 ---
cols = ["index", "cond_num", "sigma_min", "sigma_max", "v0_mps", "theta_deg", "R_m", "H_m"]
print(f"\nTop-{top_k} worst-conditioned points:")
print(topk[cols].to_string(index=False))

# 不可逆点清单导出
ill_df = df.loc[~df["invertible"], ["v0_mps","theta_deg","R_m","H_m","sigma_min","sigma_max","cond_num","log10_cond"]]
ill_csv = "ill_conditioned_points.csv"
ill_df.to_csv(ill_csv, index=False)
print(f"\nSaved ill-conditioned points to: {ill_csv}")


## 2.4 灵敏度与误差传播

对于一个候选分组 \(y\) 与自变量集合 \(x\)，显函数的敏感性矩阵可写为：

$$
\frac{\partial y}{\partial x} = - J_y^{-1} J_x, \quad
J_x = \frac{\partial F}{\partial x}.
$$

用途：  
- 变量灵敏度排序：比较各列范数 \(\|\partial y/\partial x_j\|\)。  
- 误差传播：  
  \[
  \Sigma_y \approx \Big(\frac{\partial y}{\partial x}\Big) \Sigma_x \Big(\frac{\partial y}{\partial x}\Big)^\top
  \]
  由此量化输入误差对输出的影响。

---

## 2.5 数据结构与实现

与其仅存一个二维矩阵（样本 × 分组），更适合使用 **长表 / 三维张量** 来记录各项指标：  

- 索引：\((\text{sample } i, \ \text{y-block } p)\)  
- 字段：\(\sigma_{\min}, \ \kappa, \ \operatorname{sign}\det, \ \| \partial y/\partial x\|\) 各种范数、关键导数条目、残差 \(\|F\|\) 等。  

这样方便后续分区、聚类和可视化。

---

## 2.6 分区与拼接

在每个候选分组下，可以对 \(\kappa\) 或 \(\sigma_{\min}\) 的分布做连通域分析，得到局部适用范围。  
不同的分组覆盖不同区域时，将它们拼接起来，形成完整的 **显函数贴片马赛克**。  

---

## 2.7 注意事项与常见陷阱

- **过/欠定系统**：仅在 \(|y|=m\) 时才能判定可逆性。否则需改用伪逆与最小二乘。  
- **行列式误导**：\(\det\) 极易受量纲缩放与数值误差影响。应优先看奇异值与条件数。  
- **离解流形太远**：若 \(\|F\|\) 太大，即便 Jacobian 可逆，也不代表存在物理解。  
- **采样不足**：奇异点/转捩区需加密采样，否则分区边界抖动严重。  

---

✅ 总结：  
你的“行=样本点、列=因变量分组、单元格存指标”的扫描方式是正确的。  
改进点在于：  
- 列应表示 **y-block 分组** 而非 \(\delta y\)；  
- 指标需用 **奇异值与条件数** 替代单纯的 \(\det\)；  
- 加上灵敏度矩阵与误差传播分析，结果更稳健。  

最终，这一步为 **显函数贴片的拼接与分区** 提供了定量依据。


In [2]:
# ===============================================
# Implicit-function existence & stability screening
# (General y/x dimensions; projectile motion example)
# ===============================================

import numpy as np
import pandas as pd
import sympy as sp
import matplotlib.pyplot as plt
from itertools import combinations
from numpy.linalg import svd, matrix_rank, pinv, norm

# -----------------------------
# 0) 可调参数
# -----------------------------
N = 300
g_val = 9.81
v0_min, v0_max = 5.0, 60.0
th_min_deg, th_max_deg = 5.0, 85.0
seed = 42

ADD_NOISE = False
noise_R = 0.5
noise_H = 0.2

SAVE_CSV   = False
MAKE_PLOTS = False

# —— 关键：设置 y 的维度（可选 1/2/3）——
Y_DIM = 2   # 例：设为 1 则 y 取 1 个变量；设为 3 则 y 取 3 个变量

# 数值阈值
EPS_FULLRANK = 1e-12  # 判定满秩的奇异值下限

rng = np.random.default_rng(seed)

# -----------------------------
# 1) 生成样本（无阻力抛体）
# -----------------------------
v0 = rng.uniform(v0_min, v0_max, size=N)
theta_deg = rng.uniform(th_min_deg, th_max_deg, size=N)
theta = np.deg2rad(theta_deg)

R = (v0**2) * np.sin(2 * theta) / g_val
H = (v0**2) * (np.sin(theta)**2) / (2 * g_val)

if ADD_NOISE:
    R = R + rng.normal(0, noise_R, size=N)
    H = H + rng.normal(0, noise_H, size=N)

df = pd.DataFrame({
    "v0_mps": v0,
    "theta_deg": theta_deg,
    "R_m": R,
    "H_m": H,
    "g_mps2": g_val
})
if SAVE_CSV:
    df.to_csv("ift_projectile_samples.csv", index=False)

# -----------------------------
# 2) 符号构造 F 及总雅可比 J_all（对 [R,H,v0,th]）
# -----------------------------
R_s, H_s, v0_s, th_s, g_s = sp.symbols('R H v0 th g', real=True)

F1 = R_s - (v0_s**2)*sp.sin(2*th_s)/g_s
F2 = H_s - (v0_s**2)*(sp.sin(th_s)**2)/(2*g_s)
F  = sp.Matrix([F1, F2])              # m=2 维

all_vars = [R_s, H_s, v0_s, th_s]     # 4 个候选变量的顺序
var_names = ['R', 'H', 'v0', 'th']

J_all = F.jacobian(all_vars)          # 2x4
J_func = sp.lambdify((R_s, H_s, v0_s, th_s, g_s), J_all, 'numpy')

m = F.shape[0]                        # 方程数（本例 m=2）

# -----------------------------
# 3) 生成所有候选 y-block（大小 = Y_DIM）
# -----------------------------
assert 1 <= Y_DIM <= 3, "Y_DIM 只能为 1/2/3（因为总共有4个变量，且需留出x）"
candidate_y_blocks = list(combinations(range(4), Y_DIM))

# -----------------------------
# 4) 评估函数（支持任意 y/x 维度）
# -----------------------------
def evaluate_block_general(y_idx_tuple, df, eps_rank=EPS_FULLRANK):
    """
    对任意 y/x 维度的组合进行诊断：
    - 计算 Jy (m x p), Jx (m x n)
    - 计算 SVD 奇异值、rank
    - 若 p==m 且 rank(Jy)==m：满足经典 IFT（局部唯一可解）
    - 否则：给出广义数值诊断（不宣称 IFT 可解）
    - 稳定性度量统一使用伪逆：
        * smin, smax, cond2 = smax/smin（smin<eps → cond=inf）
        * ||Jy^+||_2 = 1/smin（若 smin<eps → inf）
        * || -Jy^+ Jx ||_2
    返回：
      diag_df: 含逐样本诊断
      summary: 统计摘要（用于排序/比较）
    """
    y_idx = list(y_idx_tuple)
    x_idx = [i for i in range(4) if i not in y_idx]

    y_names = [var_names[i] for i in y_idx]
    x_names = [var_names[i] for i in x_idx]

    rows = []
    # 收集统计
    fullrank_flags = []     # 是否 rank(Jy)=min(m,p)
    ift_flags = []          # 是否满足“经典 IFT”（仅当 p==m 且 rank=m）

    cond_list = []
    smin_list, smax_list = [], []
    pinv_norm_list = []
    amp_list = []

    for i in range(len(df)):
        R0 = float(df.loc[i, 'R_m'])
        H0 = float(df.loc[i, 'H_m'])
        v00 = float(df.loc[i, 'v0_mps'])
        th0 = float(np.deg2rad(df.loc[i, 'theta_deg']))
        g0  = float(df.loc[i, 'g_mps2'])

        J = np.array(J_func(R0, H0, v00, th0, g0), dtype=float)  # 2x4
        Jy = J[:, y_idx]   # m x p
        Jx = J[:, x_idx]   # m x n

        # SVD (Jy) -> 奇异值 s1>=...>=sk
        try:
            U, svals, Vt = svd(Jy, full_matrices=False)
        except np.linalg.LinAlgError:
            svals = np.array([])
        rank_Jy = int(np.sum(svals > eps_rank))
        fullrank = (rank_Jy == min(Jy.shape))
        fullrank_flags.append(fullrank)

        # 经典 IFT 条件：p==m 且 rank(Jy)==m
        ift_ok = (Jy.shape[1] == m) and (rank_Jy == m)
        ift_flags.append(ift_ok)

        # 条件数、最小/最大奇异值
        if svals.size > 0:
            smax = float(np.max(svals))
            smin = float(np.min(svals))
            cond2 = np.inf if smin <= eps_rank else (smax / smin)
            pinv_norm = np.inf if smin <= eps_rank else (1.0 / smin)  # ||Jy^+||_2
        else:
            smax, smin, cond2, pinv_norm = np.nan, np.nan, np.inf, np.inf

        # 放大因子：|| -Jy^+ Jx ||_2
        try:
            Jy_pinv = pinv(Jy)      # 任意形状都可
            amp = norm(-Jy_pinv @ Jx, 2)
        except np.linalg.LinAlgError:
            amp = np.inf

        smax_list.append(smax)
        smin_list.append(smin)
        cond_list.append(cond2)
        pinv_norm_list.append(pinv_norm)
        amp_list.append(amp)

        rows.append({
            'sample': i,
            'R': R0, 'H': H0, 'v0': v00, 'th(rad)': th0,
            'rank(Jy)': rank_Jy,
            'full_rank(Jy)': fullrank,
            'IFT_applicable(square_and_full_rank)': ift_ok,  # 仅 p==m, rank==m 时为 True
            'smax(Jy)': smax, 'smin(Jy)': smin, 'cond2(Jy)': cond2,
            '||Jy^+||_2': pinv_norm,
            '||-Jy^+ Jx||_2': amp
        })

    diag_df = pd.DataFrame(rows)

    def finite_pct(arr, q):
        arr = np.array([a for a in arr if np.isfinite(a)], dtype=float)
        if arr.size == 0: return np.nan
        return float(np.percentile(arr, q))

    summary = {
        'y_block': tuple(y_names),
        'x_block': tuple(x_names),
        'y_dim': len(y_names),
        # 统计：满秩率、IFT 可用率
        'fullrank_rate(Jy)': float(np.mean(fullrank_flags)),
        'IFT_rate(square_and_full_rank)': float(np.mean(ift_flags)),
        # 条件数/奇异值/伪逆范数/放大因子（分位数）
        'median_cond2': finite_pct(cond_list, 50),
        'p90_cond2':    finite_pct(cond_list, 90),
        'median_smin':  finite_pct(smin_list, 50),
        'p10_smin':     finite_pct(smin_list, 10),
        'median_||Jy^+||_2': finite_pct(pinv_norm_list, 50),
        'p90_||Jy^+||_2':    finite_pct(pinv_norm_list, 90),
        'median_amp':        finite_pct(amp_list, 50),
        'p90_amp':           finite_pct(amp_list, 90),
    }
    return diag_df, summary

# -----------------------------
# 5) 全局扫描并排序
# -----------------------------
summaries = []
diagnostics_by_block = {}

for y_pair in candidate_y_blocks:
    diag_df, summary = evaluate_block_general(y_pair, df)
    summaries.append(summary)
    diagnostics_by_block[summary['y_block']] = diag_df

summary_df = pd.DataFrame(summaries)

# 排序策略：
# 1) IFT 可用率（只有 y_dim==m 时可能>0）优先
# 2) 再看 cond2 中位数（越小越好）
# 3) 再看放大因子 p90（越小越好）
summary_df = summary_df.sort_values(
    by=['IFT_rate(square_and_full_rank)', 'median_cond2', 'p90_amp'],
    ascending=[False, True, True]
).reset_index(drop=True)

# 可选：保存
if SAVE_CSV:
    summary_df.to_csv("implicit_block_screen_general_summary.csv", index=False)
    top_block = tuple(summary_df.loc[0, 'y_block'])
    diagnostics_by_block[top_block].to_csv(
        f"diagnostics_top_block_{'_'.join(top_block)}.csv", index=False
    )

# 可选：简单绘图（对 Top 组合）
if MAKE_PLOTS:
    top_block = tuple(summary_df.loc[0, 'y_block'])
    top_diag = diagnostics_by_block[top_block]
    finite_cond = top_diag['cond2(Jy)'][np.isfinite(top_diag['cond2(Jy)'])]
    finite_amp  = top_diag['||-Jy^+ Jx||_2'][np.isfinite(top_diag['||-Jy^+ Jx||_2'])]

    plt.figure()
    plt.hist(np.log10(finite_cond), bins=30)
    plt.xlabel('log10 cond2(Jy)')
    plt.ylabel('count')
    plt.title(f'Conditioning for y={top_block}')
    plt.show()

    plt.figure()
    plt.hist(np.log10(finite_amp), bins=30)
    plt.xlabel('log10 ||-Jy^+ Jx||_2')
    plt.ylabel('count')
    plt.title(f'Amplification ||-Jy^+ Jx|| for y={top_block}')
    plt.show()

# 浏览结果（Notebook 中直接显示）
summary_df.head()


,y_block,x_block,y_dim,fullrank_rate(Jy),IFT_rate(square_and_full_rank),median_cond2,p90_cond2,median_smin,p10_smin,median_||Jy^+||_2,p90_||Jy^+||_2,median_amp,p90_amp
0,"(R, H)","(v0, th)",2,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,112.003841,381.452539
1,"(H, v0)","(R, th)",2,1.0,1.0,5.477179,10.596539,0.953009,0.644041,1.049310,1.552748,70.911449,298.445302
2,"(R, v0)","(H, th)",2,1.0,1.0,22.811350,65.736231,0.242430,0.052023,4.124916,19.222648,216.158395,599.379913
3,"(v0, th)","(R, H)",2,1.0,1.0,42.363186,720.705353,2.464930,0.113530,0.405693,8.809491,0.405693,8.809491
4,"(H, th)","(R, v0)",2,1.0,1.0,152.472621,440.207544,0.953448,0.452976,1.048827,2.208880,3.103615,13.567313


**Step 6：**  **Jacobian 矩阵的敏感性** 

$$S = -J^{-1}_y J_x$$

它给出“自变量微小变化 δx 对因变量 δy 的线性响应”：$\delta{y} = S \delta{x}$

In [ ]:
# --- Step 6: 敏感性矩阵 S = - Jy^{-1} Jx ---

# 先切出 Jx（这里按 y=[v0, th], x=[R, H]）
idx_y = [2, 3]          # 因变量 v0, th
idx_x = [0, 1]          # 自变量 R, H
Jy = J_all[:, idx_y]    # 2x2
Jx = J_all[:, idx_x]    # 2x2

# 符号式敏感性：用 LUsolve 避免显式逆，更稳
S_sym = - Jy.LUsolve(Jx)        # 等价于 -(Jy.inv() * Jx)，但数值更稳
S_sym = sp.simplify(S_sym)

print("\nSymbolic Sensitivity S = -Jy^{-1} Jx:")
sp.pprint(S_sym)

# 数值版：lambdify 并代入基点
S_func  = sp.lambdify((R_s, H_s, v0_s, th_s, g_s), S_sym, 'numpy')
Jy_func = sp.lambdify((R_s, H_s, v0_s, th_s, g_s), Jy,     'numpy')  # 若你还想复用 Jy 数值
Jx_func = sp.lambdify((R_s, H_s, v0_s, th_s, g_s), Jx,     'numpy')

S_num  = np.array(S_func(R_val, H_val, v0_val, th_val, g_val), dtype=float)
Jy_num = np.array(Jy_func(R_val, H_val, v0_val, th_val, g_val), dtype=float)
Jx_num = np.array(Jx_func(R_val, H_val, v0_val, th_val, g_val), dtype=float)

print("\nS (numeric, 2x2) with rows [dv0, dth], cols [dR, dH]:")
print(S_num)

# --- 可选：归一化敏感性（相对变化），便于比较不同量纲/尺度 ---
# S_rel[i,j] = (∂y_i/∂x_j) * (x_j / y_i)
x_vec = np.array([R_val, H_val], dtype=float)     # 自变量基点
y_vec = np.array([v0_val, th_val], dtype=float)   # 因变量基点
S_rel_num = S_num * x_vec[None, :] / y_vec[:, None]

print("\nS_rel (relative sensitivity) where S_rel[i,j] = (∂y_i/∂x_j)*(x_j/y_i):")
print(S_rel_num)

# --- 可选：校验线性近似 δy ≈ S δx ---
# 举例: 让 R 增加 1%，H 不变，看看 δy 的线性预测
delta_x = np.array([0.01 * R_val, 0.0], dtype=float)
delta_y_lin = S_num @ delta_x
print("\nExample: R +1% (H fixed) → predicted [δv0, δth]:")
print(delta_y_lin)


In [ ]:
# 扫描示例

J_funcs = sp.lambdify((R_s, H_s, v0_s, th_s, g_s), J_all, 'numpy')

# 变量名称（用于展示）
var_names = ['R', 'H', 'v0', 'th']


# --- Step 4: evaluation function ---
def evaluate_block(y_idx_pair, df, eps_det=EPS_DET):
    """
    对特定的 y-block（两个变量）做存在性与稳定性评估：
    - 可解性：det(Jy) != 0
    - 稳定性：cond(Jy), ||Jy^{-1}||_2, ||-Jy^{-1} Jx||_2

    返回：
    - diag_df: 每个样本的逐点诊断
    - summary: 该组合的整体统计（用于排序）
    """
    y_idx = list(y_idx_pair)
    x_idx = [i for i in range(4) if i not in y_idx]

    y_names = [var_names[i] for i in y_idx]
    x_names = [var_names[i] for i in x_idx]

    rows = []
    det_list, cond_list, gain_list, amp_list, inv_flags = [], [], [], [], []

    for i in range(len(df)):
        # 取样本点（注意 theta 用弧度代入符号函数）
        R0 = float(df.loc[i, 'R_m'])
        H0 = float(df.loc[i, 'H_m'])
        v00 = float(df.loc[i, 'v0_mps'])
        th0 = float(np.deg2rad(df.loc[i, 'theta_deg']))
        g0  = float(df.loc[i, 'g_mps2'])

        # 2x4 雅可比矩阵
        J_all_np = np.array(J_funcs(R0, H0, v00, th0, g0), dtype=float)

        # 切出 Jy (2x2), Jx (2x2)
        Jy = J_all_np[:, y_idx]     # 对 y 的偏导
        Jx = J_all_np[:, x_idx]     # 对 x 的偏导

        # 可解性：det(Jy) != 0（仅在 2x2 情形可用）
        try:
            det_val = float(np.linalg.det(Jy))
        except LinAlgError:
            det_val = np.nan

        is_invertible = np.isfinite(det_val) and (abs(det_val) > eps_det)

        # 稳定性指标：cond(Jy), ||Jy^{-1}||_2, ||-Jy^{-1} Jx||_2
        if is_invertible:
            try:
                Jy_inv = inv(Jy)
                kappa = cond(Jy)                  # 2-范数条件数
                gain  = norm(Jy_inv, 2)           # ||Jy^{-1}||_2：F扰动→y扰动
                amp   = norm(-Jy_inv @ Jx, 2)     # ||-Jy^{-1}Jx||_2：x扰动→y扰动
            except LinAlgError:
                kappa, gain, amp = np.inf, np.inf, np.inf
        else:
            kappa, gain, amp = np.inf, np.inf, np.inf

        rows.append({
            'sample': i,
            'R': R0, 'H': H0, 'v0': v00, 'th(rad)': th0,
            'det(Jy)': det_val,
            'invertible': is_invertible,
            'cond(Jy)': kappa,
            '||Jy^{-1}||_2': gain,
            '||-Jy^{-1}Jx||_2': amp
        })

        det_list.append(det_val)
        cond_list.append(kappa)
        gain_list.append(gain)
        amp_list.append(amp)
        inv_flags.append(is_invertible)

    diag_df = pd.DataFrame(rows)

    # 统计函数（忽略 inf / nan）
    def finite_percentile(arr, q):
        arr = np.array([a for a in arr if np.isfinite(a)], dtype=float)
        if arr.size == 0: return np.nan
        return float(np.percentile(arr, q))

    summary = {
        'y_block': tuple(y_names),
        'x_block': tuple(x_names),
        'invertible_rate': float(np.mean(inv_flags)),
        'median_cond': finite_percentile(cond_list, 50),
        'p90_cond':    finite_percentile(cond_list, 90),
        'median_gain': finite_percentile(gain_list, 50),
        'p90_gain':    finite_percentile(gain_list, 90),
        'median_amp':  finite_percentile(amp_list, 50),
        'p90_amp':     finite_percentile(amp_list, 90),
    }
    return diag_df, summary

# --- Step 5: evaluate all candidate blocks & rank ---
summaries = []
diagnostics = {}  # key: y_block(tuple of names) -> per-sample diagnostics

for y_pair in candidate_y_blocks:
    diag_df, summary = evaluate_block(y_pair, df, eps_det=EPS_DET)
    summaries.append(summary)
    diagnostics[summary['y_block']] = diag_df

summary_df = pd.DataFrame(summaries)

# 排序准则：可解率高优先 → 条件数中位数低 → 放大因子 p90 低
summary_df = summary_df.sort_values(
    by=['invertible_rate', 'median_cond', 'p90_amp'],
    ascending=[False, True, True]
).reset_index(drop=True)

# 可选：保存结果
if save_csv:
    summary_df.to_csv("implicit_block_screen_summary.csv", index=False)
    # 保存 top 组合的逐样本诊断
    top_block = tuple(summary_df.loc[0, 'y_block'])
    diagnostics[top_block].to_csv(f"diagnostics_top_block_{'_'.join(top_block)}.csv", index=False)

# --- Step 6: quick plots (optional) ---
if make_plots:
    top_block = tuple(summary_df.loc[0, 'y_block'])
    top_diag = diagnostics[top_block]

    # 只对有限值作直方图
    finite_cond = top_diag['cond(Jy)'][np.isfinite(top_diag['cond(Jy)'])]
    finite_amp  = top_diag['||-Jy^{-1}Jx||_2'][np.isfinite(top_diag['||-Jy^{-1}Jx||_2'])]

    plt.figure()
    plt.hist(np.log10(finite_cond), bins=30)
    plt.xlabel('log10 cond(Jy)')
    plt.ylabel('count')
    plt.title(f'Conditioning distribution for y={top_block}')
    plt.show()

    plt.figure()
    plt.hist(np.log10(finite_amp), bins=30)
    plt.xlabel('log10 ||-Jy^{-1}Jx||_2')
    plt.ylabel('count')
    plt.title(f'Amplification ||-Jy^(-1)Jx|| for y={top_block}')
    plt.show()

# --- Step 7: 展示核心输出（可在Notebook中直接查看） ---
summary_df.head()